# Deep Agents

[deepagents](https://docs.langchain.com/oss/python/deepagents/overview) 可以看作是 LangChain 团队出品的 DeepResearch. 本节通过查询一个近期新闻，检验 Deep Agents 是否有主动调用搜索，获取外部信息的能力。

> 这里只是简单 quickstart 一下，更多信息大家去官网看吧！

In [1]:
# !pip install deepagents dashscope

**1）加载模型**

In [2]:
import os
import dashscope

from datetime import datetime
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from deepagents import create_deep_agent
from dashscope import Generation

# 加载模型配置
_ = load_dotenv()

# 为灵积配置 api_key
dashscope.api_key=os.getenv("DASHSCOPE_API_KEY")

# 加载模型
llm = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    model="qwen3-max",
)

**2）创建搜索工具**

[DDGS](https://github.com/deedy5/ddgs) 是我找到的一款免费的、无需 `API_KEY` 即可直接使用的网络搜索服务。我们用它替代官方推荐的 `Tavily`.

In [3]:
@tool
def dashscope_search(query: str) -> str:
    """
    使用夸克搜索 API 搜索互联网信息。
    """
    response = Generation.call(
        model='qwen-max',
        prompt=query,
        enable_search=True,
        result_format='message'
    )

    if response.status_code == 200:
        return response.output.choices[0].message.content
    else:
        return (
            "Search failed with status code: "
            f"{response.status_code}, message: {response.message}"
        )

**3）创建深度代理**

In [4]:
# System prompt to steer the agent to be an expert researcher
research_instructions = """你是一名资深研究员。你的工作是进行全面深入的研究，并撰写一份精炼的报告。

你可以使用互联网搜索工具作为获取信息的主要方式。

## `dashscope_search`

使用该工具对指定查询进行互联网搜索。
"""

@tool
def get_today_date() -> str:
    """获取今天的日期"""
    return datetime.now().strftime("%Y-%m-%d")

agent = create_deep_agent(
    model=llm,
    tools=[dashscope_search, get_today_date],
    system_prompt=research_instructions
)

**4）运行 Agent**

我们的问题是：

```
新上任的玻利维亚总统是谁？请介绍一下这位总统。
```

因为昨天（2025 年 11 月 8 日）玻利维亚总统 `罗德里戈・帕斯・佩雷拉` 刚刚上任，这条信息肯定不在预训练数据里。可以用这个问题验证 Deep Agents 是否真的联网了。

In [5]:
result = agent.invoke({"messages": [
    {"role": "user", "content": "新上任的玻利维亚总统是谁？请介绍一下这位总统。"}
]})

# 最终回复
print(result["messages"][-1].content)

根据我收集到的信息，我可以为您介绍新上任的玻利维亚总统：

# 玻利维亚新任总统：罗德里戈·帕斯·佩雷拉

## 基本信息
- **全名**：罗德里戈·帕斯·佩雷拉（Rodrigo Paz Pereira）
- **出生**：1967年9月22日出生于西班牙加利西亚的圣地亚哥德孔波斯特拉
- **就职时间**：2025年11月8日
- **任期**：5年
- **所属政党**：中右翼基督教民主党

## 教育背景
- 美国美利坚大学经济学与国际关系学士学位
- 美国美利坚大学政治管理硕士学位

## 政治背景与家族历史
罗德里戈·帕斯·佩雷拉出身于玻利维亚著名的政治世家：
- 外曾祖父维克托·帕斯·埃斯特恩索罗是玻利维亚现代化的重要推动者之一
- 父亲海梅·帕斯·萨莫拉曾在1989年至1993年间担任玻利维亚总统，推行温和的社会民主路线

在成为总统之前，帕斯·佩雷拉已积累了丰富的政治经验，曾担任塔里哈市市长及参议员。

## 2025年总统选举
- 在第二轮投票中以54.5%的支持率胜出
- 击败了右翼"自由"联盟候选人豪尔赫·基罗加（获得45.5%选票）
- 这次选举标志着玻利维亚政治从近20年的左翼执政转向右翼

## 政策立场
1. **外交政策**：
   - 就职后立即宣布与美国全面恢复大使级外交关系
   - 强调"基于务实主义与对外开放的外交政策转型"
   - 被视为采取了一条既亲美也友华的道路，但可能更倾向于加强与美国的关系

2. **经济政策**：
   - 计划通过开放经济吸引更多私人投资促进国家发展
   - 承诺继续支持社会福利项目
   - 可能会审查甚至推翻之前与中国企业签订的一些锂矿合作合同
   - 寻求将锂资源优先供应给美国企业

3. **政治定位**：
   - 被视为能够在不同政治势力之间取得平衡的中间派人物
   - 以技术官僚的专业性和务实性为竞选特色

值得注意的是，尽管帕斯·佩雷拉表达了向美国靠拢的意愿，但他也强调了务实主义的重要性，这意味着玻利维亚仍有可能维持与中国的部分合作关系，特别是在经济利益驱动下。


In [6]:
# 思考过程
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

新上任的玻利维亚总统是谁？请介绍一下这位总统。
================================== Ai Message ==================================

我需要查找新上任的玻利维亚总统的信息。让我通过互联网搜索来获取最新信息。
Tool Calls:
  dashscope_search (call_4c1cb6a4432346e2a83f9972)
 Call ID: call_4c1cb6a4432346e2a83f9972
  Args:
    query: 新上任的玻利维亚总统
================================= Tool Message =================================
Name: dashscope_search

新上任的玻利维亚总统是罗德里戈·帕斯·佩雷拉。他于2025年11月8日在玻利维亚行政首都拉巴斯宣誓就职，开启了为期5年的任期。帕斯在选举中代表中右翼基督教民主党，并在第二轮投票中以54.5%的支持率胜出。此外，据报道，帕斯上任后宣布与美国全面恢复外交关系。
================================== Ai Message ==================================

根据搜索结果，新上任的玻利维亚总统是**罗德里戈·帕斯·佩雷拉**（Rodrigo Paz Pereira）。

让我进一步了解这位总统的背景和政策主张：
Tool Calls:
  dashscope_search (call_3d8381b4cd094f9f96d04eb1)
 Call ID: call_3d8381b4cd094f9f96d04eb1
  Args:
    query: 罗德里戈·帕斯·佩雷拉 玻利维亚总统 背景
================================= Tool Message =================================
Name: dashscope_search
